In [3]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [5]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#Define a new graph
workflow = StateGraph(state_schema=MessagesState)


#Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


#Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = {"configurable": {"thread_id": "test_3"}}

In [13]:
from langchain.prompts import ChatPromptTemplate

system_prompt = """You are a friendly inquisitive chatbot. Your goal is to learn about the user by asking questions naturally.  
Gather the following details (prioritize missing info):  
- Name  
- Profession  
- Hobbies  
- Favorite books/movies  
- Life goals  
- Dreams and their 'Bucket List'

**Rules**:
1. Be conversational and friendly.
2. Ask only one question at a time if asking a question.
3. Only ask for missing information.
4. Never repeat questions.
5. Adapt based on their responses.
6. Limit questions and conversation to a maximum of three lines.
7. Don't deviate to much from your question and answers.
"""

# Simplified prompt (no MessagesPlaceholder needed)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    # User messages will be injected by LangGraph's state["messages"]
])

In [ ]:
# First turn
response1 = app.invoke(
    {"messages": [HumanMessage(content="Hi!")]},
    config=config
)
print(response1["messages"][-1].content)  
# Output: "Nice to meet you! What's your name?"

# Second turn (LangGraph automatically includes past messages)
response2 = app.invoke(
    {"messages": [HumanMessage(content="I'm Alex.")]},
    config=config
)
print(response2["messages"][-1].content)  
# Output: "Got it, Alex! What do you do for work?"

Hi Alex! Still just saying hi? 😊 Let me know if you have any questions or if there's anything I can do for you. I'm here to help!
Okay, Alex. I remember you're Alex. Is there anything you'd like to talk about or ask me, Alex?


In [ ]:
import requests
from flask import Flask, request, jsonify
from langchain.schema import HumanMessage

jupyter_app = Flask(__name__)



@jupyter_app.route('/trigger-feedback', methods=['POST'])
def trigger_feedback():
    try:
        data = request.get_json()
        if not data or 'answer' not in data:
            return jsonify({'error': 'Missing answer in request'}), 400
        
        user_message = data['answer']
        
        # Process message through LangGraph
        response_to_user = app.invoke(
            {"messages": [HumanMessage(content=user_message)]},
            config=config
        )
        
        # Extract the last message content
        feedback = response_to_user["messages"][-1].content
        
        return jsonify({
            'feedback': feedback,
            'status': 'success'
        })
        
    except Exception as e:
        print(f"Error generating feedback: {str(e)}", flush=True)
        return jsonify({'error': str(e)}), 500
        
# Run the server in a separate thread to avoid blocking the notebook
from threading import Thread
def run_jupyter_server():
    jupyter_app.run(host='127.0.0.1', port=5001, debug=False)

if __name__ == '__main__':
    # Start the server in a thread
    server_thread = Thread(target=run_jupyter_server)
    server_thread.daemon = True  # Stops when notebook stops
    server_thread.start()
    print("Jupyter question generator server running on port 5001...")

Jupyter question generator server running on port 5001...


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [05/Apr/2025 19:56:14] "POST /trigger-feedback HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 19:56:19] "POST /trigger-feedback HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 19:59:31] "POST /trigger-feedback HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 19:59:39] "POST /trigger-feedback HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2025 19:59:43] "POST /trigger-feedback HTTP/1.1" 200 -
